In [1]:
import sqlite3
import pandas as pd

In [2]:
# Connect to SQLite (create a database in memory)
conn = sqlite3.connect(':memory:') 

# Load CSV files into pandas DataFrames
sales_data = pd.read_csv('Sales_Data.csv')
glance_views = pd.read_csv('Glance_Views.csv')

# Load DataFrames into SQLite tables
sales_data.to_sql('Sales_Data', conn, index=False, if_exists='replace')
glance_views.to_sql('Glance_Views', conn, index=False, if_exists='replace')

40745

In [3]:
sales_data.head() 
glance_views.head()  

,Unnamed: 0,SKU_NAME,FEED_DATE,VIEWS,UNITS
0,0,B1212:PZ:V,2019-05-01,455.0,16.0
1,1,B1212:PZ:V,2019-05-02,478.0,12.0
2,2,B1212:PZ:V,2019-05-03,681.0,42.0
3,3,B1212:PZ:V,2019-05-04,662.0,70.0
4,4,B1212:PZ:V,2019-05-05,568.0,33.0


In [4]:
sales_data.isnull().sum()  # Check for missing values

Unnamed: 0           0
SKU_NAME             0
FEED_DATE            0
CATEGORY             0
SUB_CATEGORY         0
ORDERED_REVENUE      0
ORDERED_UNITS        0
REP_OOS            689
dtype: int64

In [5]:
sales_data['REP_OOS'].fillna(0, inplace=True)  # Replace with zero

In [6]:
sales_data.isnull().sum()

Unnamed: 0         0
SKU_NAME           0
FEED_DATE          0
CATEGORY           0
SUB_CATEGORY       0
ORDERED_REVENUE    0
ORDERED_UNITS      0
REP_OOS            0
dtype: int64

## Question 1: Identify the most expensive SKU, on average, over the entire time period

In [7]:
query = '''
    SELECT SKU_Name, AVG(Ordered_Revenue / Ordered_Units) AS Avg_Price
    FROM Sales_Data
    GROUP BY SKU_Name
    ORDER BY Avg_Price DESC
    LIMIT 1;
'''

result = pd.read_sql(query, conn)
print(result)

     SKU_NAME    Avg_Price
0  C03CBL[721  1151.858727


## Question 2: What % of SKUs have generated some revenue in this time period?

In [8]:
query = '''
    SELECT (COUNT(DISTINCT SKU_Name) * 100.0) / 
       (SELECT COUNT(DISTINCT SKU_Name) FROM Sales_Data) AS Revenue_Generating_SKU_Percentage
FROM Sales_Data
WHERE Ordered_Revenue > 0;
'''

result = pd.read_sql(query, conn)
print(result)

   Revenue_Generating_SKU_Percentage
0                          78.709677


### SKUs That Stopped Selling After July:

In [9]:
query = '''
    SELECT DISTINCT SKU_Name
FROM Sales_Data
WHERE strftime('%m', Feed_date) <= '07'
AND SKU_Name NOT IN (
    SELECT DISTINCT SKU_Name
    FROM Sales_Data
    WHERE strftime('%m', Feed_date) > '07'
      AND Ordered_Revenue > 0
);
'''

result = pd.read_sql(query, conn)
print(result)

       SKU_NAME
0    B12020KBUI
1    C02228YPPT
2    D1000D20S:
3    C1027E31J0
4    D211PEL2IE
..          ...
134  C29LCOGDHZ
135  C17NEDU7P[
136  C28S6S9HS[
137  C07S8PDLZX
138  C19T:CGV3L

[139 rows x 1 columns]


## Question 3: Somewhere in this timeframe, there was a Sale Event. Identify the dates.

In [10]:
query = '''
    SELECT Feed_date, SUM(Ordered_Revenue) AS Total_Revenue
FROM Sales_Data
GROUP BY Feed_date
ORDER BY Total_Revenue DESC
LIMIT 5;
'''

result = pd.read_sql(query, conn)
print(result)

    FEED_DATE  Total_Revenue
0  2019-07-15     5158848.46
1  2019-07-16     3105376.74
2  2019-06-20     1586367.45
3  2019-07-26     1266935.31
4  2019-06-26      926478.82


## Question 4:  Does having a sale event cannibalize sales in the immediate aftermath? Highlighting a few examples would suffice 

In [11]:
query = '''
    WITH Date_Revenue AS (
    SELECT Feed_date, SUM(Ordered_Revenue) AS Total_Revenue
    FROM Sales_Data
    GROUP BY Feed_date
),
Max_Revenue AS (
    SELECT MAX(Total_Revenue) AS Max_Total_Revenue
    FROM Date_Revenue
),
Sale_Dates AS (
    SELECT Feed_date
    FROM Date_Revenue
    WHERE Total_Revenue = (SELECT Max_Total_Revenue FROM Max_Revenue)
)
SELECT sd.Feed_date, sd.SKU_Name, sd.Ordered_Revenue, 
       (SELECT AVG(sd2.Ordered_Revenue)
        FROM Sales_Data sd2
        WHERE sd2.Feed_date > sd.Feed_date
        ORDER BY sd2.Feed_date
        LIMIT 3) AS Post_Sale_Revenue
FROM Sales_Data sd
JOIN Sale_Dates s ON sd.Feed_date = s.Feed_date
ORDER BY sd.Feed_date;


'''

result = pd.read_sql(query, conn)
print(result)

      FEED_DATE    SKU_NAME  ORDERED_REVENUE  Post_Sale_Revenue
0    2019-07-15  B1212:PZ:V          1789.80        2187.648591
1    2019-07-15  B111S;CCLC          8187.55        2187.648591
2    2019-07-15  D102VX\R52         31657.76        2187.648591
3    2019-07-15  C120[H:8NV          1674.40        2187.648591
4    2019-07-15  B223F54NLG          3329.14        2187.648591
..          ...         ...              ...                ...
329  2019-07-15  D28NTVMU32          1349.91        2187.648591
330  2019-07-15  D07QEFDK\E          -519.96        2187.648591
331  2019-07-15  D08RJOM3XB             0.00        2187.648591
332  2019-07-15  C08QQJNQR4             0.00        2187.648591
333  2019-07-15  D28QU2Q7[:           700.32        2187.648591

[334 rows x 4 columns]


## Question 5: In each category, find the subcategory that has grown slowest relative to the category it is present in. If you were handling the entire portfolio, which of these subcategories would you be most concerned with?

In [12]:
query = '''
    WITH Growth_Rate AS (
    SELECT Category, Sub_Category, 
           (MAX(Ordered_Revenue) - MIN(Ordered_Revenue)) / MIN(Ordered_Revenue) AS Growth
    FROM Sales_Data
    GROUP BY Category, Sub_Category
)
SELECT Category, Sub_Category
FROM Growth_Rate
WHERE Growth = (SELECT MIN(Growth) FROM Growth_Rate);

'''

result = pd.read_sql(query, conn)
print(result)

      Category  Sub_Category
0  1000 Inputs  1005 Webcams


## Question 6: Highlight any anomalies/mismatches in the data that you see, if any. (In terms of data quality issues)

In [13]:
# Ordered_Units = 0 but Ordered_Revenue > 0

query = '''
    SELECT *
FROM Sales_Data
WHERE Ordered_Units = 0 AND Ordered_Revenue > 0;

'''

result = pd.read_sql(query, conn)
print(result)

# Out of Stock but Revenue Was Generated:

query = '''
    SELECT *
FROM Sales_Data
WHERE Rep_OOS > 0 AND Ordered_Revenue > 0;

'''

result = pd.read_sql(query, conn)
print(result)

    Unnamed: 0    SKU_NAME   FEED_DATE                     CATEGORY  \
0         2397  C124PR58CZ  2019-08-17                  1000 Inputs   
1         4028  B215GHP7Z7  2019-08-28                  1000 Inputs   
2         4788  D014[Y8YEZ  2019-06-23                  1000 Inputs   
3         4878  C024ZE\C:9  2019-05-21                  1000 Inputs   
4         6222  B027LTJDHL  2019-08-31                  1000 Inputs   
..         ...         ...         ...                          ...   
89       38575  B08MD1RCIY  2019-06-22              5300 Headphones   
90       38816  B17NJ;4CN7  2019-06-21                  1000 Inputs   
91       39650  C09OGREK9L  2019-07-17  5000 Portable Media Players   
92       39652  C09OGREK9L  2019-07-19  5000 Portable Media Players   
93       40932  D28QU2Q7[:  2019-05-21                  1000 Inputs   

                       SUB_CATEGORY  ORDERED_REVENUE  ORDERED_UNITS  REP_OOS  
0                         1002 Mice             9.11            0.0 

## Question 7: For SKU Name C120[H:8NV, discuss whether Unit Conversion (Units/Views) is affected by Average Selling Price.

In [14]:
query = '''
    WITH Conversion AS (
    SELECT gv.SKU_Name, gv.Feed_date, gv.Units, gv.Views, 
           sd.Ordered_Revenue, sd.Ordered_Units,
           (gv.Units * 1.0) / gv.Views AS Unit_Conversion,
           (sd.Ordered_Revenue * 1.0) / sd.Ordered_Units AS Avg_Selling_Price
    FROM Glance_Views gv
    JOIN Sales_Data sd ON gv.SKU_Name = sd.SKU_Name AND gv.Feed_date = sd.Feed_date
    WHERE gv.SKU_Name = 'C120[H:8NV]'
)
SELECT Unit_Conversion, Avg_Selling_Price
FROM Conversion;


'''

result = pd.read_sql(query, conn)
print(result)

Empty DataFrame
Columns: [Unit_Conversion, Avg_Selling_Price]
Index: []


In [15]:
query = '''
SELECT * 
FROM Sales_Data
WHERE SKU_Name = 'C120[H:8NV]';

'''

result = pd.read_sql(query, conn)
print(result)

Empty DataFrame
Columns: [Unnamed: 0, SKU_NAME, FEED_DATE, CATEGORY, SUB_CATEGORY, ORDERED_REVENUE, ORDERED_UNITS, REP_OOS]
Index: []


## Cannibalization Effect After Sale Event

In [16]:
import pandas as pd
from scipy import stats

# Load Sales Data
sales_data = pd.read_csv('Sales_Data.csv')

# Convert 'Feed_date' to datetime
sales_data['FEED_DATE'] = pd.to_datetime(sales_data['FEED_DATE'])

# Identify the sale event date (let's assume you identified '2024-05-10' as the sale date)
sale_event_date = pd.to_datetime('2024-05-10')

# Filter data 7 days before and after the sale event
pre_sale_data = sales_data[sales_data['FEED_DATE'] < sale_event_date].tail(7)
post_sale_data = sales_data[sales_data['FEED_DATE'] > sale_event_date].head(7)

# Calculate total sales before and after the event
pre_sale_revenue = pre_sale_data['ORDERED_REVENUE'].sum()
post_sale_revenue = post_sale_data['ORDERED_REVENUE'].sum()

# Perform a t-test to see if there is a significant difference
t_stat, p_value = stats.ttest_ind(pre_sale_data['ORDERED_REVENUE'], post_sale_data['ORDERED_REVENUE'])

print(f"T-Statistic: {t_stat}, P-Value: {p_value}")

# Check significance
if p_value < 0.05:
    print("There is a significant difference in sales before and after the sale event (possible cannibalization).")
else:
    print("No significant cannibalization effect detected after the sale event.")


T-Statistic: nan, P-Value: nan
No significant cannibalization effect detected after the sale event.


## Unit Conversion (Units/Views) vs. Average Selling Price correlation for SKU C120[H:8NV].

In [17]:
import seaborn as sns
import matplotlib.pyplot as plt

# Load Glance Views Data
glance_views = pd.read_csv('Glance_Views.csv')

# Filter data for SKU C120[H:8NV]
sku_data = sales_data[sales_data['SKU_NAME'] == 'C120[H:8NV]'].merge(glance_views, on=['FEED_DATE', 'SKU_NAME'], how='inner')

# Calculate Unit Conversion (Units/Views)
sku_data['Unit_Conversion'] = sku_data['UNITS'] / sku_data['VIEWS']

# Calculate Average Selling Price (Ordered_Revenue/Ordered_Units)
sku_data['Avg_Selling_Price'] = sku_data['ORDERED_REVENUE'] / sku_data['ORDERED_UNITS']

# Correlation analysis between Unit Conversion and Average Selling Price
correlation = sku_data['Unit_Conversion'].corr(sku_data['Avg_Selling_Price'])
print(f"Correlation between Unit Conversion and Average Selling Price: {correlation}")


Correlation between Unit Conversion and Average Selling Price: nan


In [18]:
print(sku_data[['Unit_Conversion', 'Avg_Selling_Price']].describe())

       Unit_Conversion  Avg_Selling_Price
count              0.0                0.0
mean               NaN                NaN
std                NaN                NaN
min                NaN                NaN
25%                NaN                NaN
50%                NaN                NaN
75%                NaN                NaN
max                NaN                NaN


In [19]:
sku_data = sku_data.dropna(subset=['Unit_Conversion', 'Avg_Selling_Price'])

In [20]:
correlation = sku_data['Unit_Conversion'].corr(sku_data['Avg_Selling_Price'])
print(f"Correlation between Unit Conversion and Average Selling Price: {correlation}")

Correlation between Unit Conversion and Average Selling Price: nan


In [21]:
print(sales_data['SKU_NAME'].unique())  # To check if C120[H:8NV] exists
print(glance_views['SKU_NAME'].unique())  # Check in Glance_Views if it's there


['B12020KBUI' 'D12125XVGK' 'C02228YPPT' 'B1212:PZ:V' 'D1000D20S:'
 'B111S;CCLC' 'D102VX\\R52' 'C120[H:8NV' 'C1027E31J0' 'B223F54NLG'
 'C211F62H36' 'C213I5Z84I' 'D211PEL2IE' 'B023PT;UK5' 'B123P7CEQM'
 'C0038K4UWZ' 'B112D4[DQM' 'D102FV7QM2' 'B012GU7SOL' 'B012JWTJDC'
 'D214HWSKBO' 'D023IWR\\2M' 'C013KD0YM:' 'C102ONY5WZ' 'B004RU\\O1U'
 'C214SL86Q[' 'B222RN877C' 'B1255YTFGA' 'B0148G:X2:' 'B125:PNERK'
 'C115B4DCFK' 'C204DR2PT4' 'B004FMWNKW' 'B205J5RPJQ' 'D115J4FH48'
 'D223J6HIOC' 'B213K5FIPU' 'B204N64U7W' 'C115NX\\P:U' 'D024M2YU:7'
 'C124PR58CZ' 'C104OR985T' 'D114PRFDD:' 'B213VH76GH' 'D013XAHXXL'
 'D214VCIZ\\D' 'B115WBIYPC' 'C113XCHZQZ' 'D014WCJ[VI' 'D224XBK2GB'
 'B203VBOO9C' 'D203VCOPFZ' 'C1061ST0VY' 'B2154U7FZF' 'B215GHP7Z7'
 'B124O1:Q80' 'B104NF32NV' 'D225P747LT' 'C125OVFDU1' 'B004WQ8HS6'
 'D126[CVF9J' 'D014[Y8YEZ' 'C024ZE\\C:9' 'D126DM[TMI' 'B116DUQLEB'
 'C105HUP[UO' 'B227GVQ09Q' 'B207GW2HRZ' 'D225JQ6249' 'B007LUCJZW'
 'B205LUBLYM' 'B006KSCQKJ' 'D107M38WRW' 'B017LI:T3T' 'B027LTJDHL'
 'B0

In [22]:
print(sales_data['FEED_DATE'].unique())  # Check the available dates
print(glance_views['FEED_DATE'].unique())

['2019-05-18T00:00:00.000000000' '2019-05-19T00:00:00.000000000'
 '2019-05-22T00:00:00.000000000' '2019-05-23T00:00:00.000000000'
 '2019-05-27T00:00:00.000000000' '2019-05-28T00:00:00.000000000'
 '2019-08-17T00:00:00.000000000' '2019-08-18T00:00:00.000000000'
 '2019-08-21T00:00:00.000000000' '2019-08-22T00:00:00.000000000'
 '2019-06-24T00:00:00.000000000' '2019-06-25T00:00:00.000000000'
 '2019-05-01T00:00:00.000000000' '2019-05-02T00:00:00.000000000'
 '2019-05-03T00:00:00.000000000' '2019-05-04T00:00:00.000000000'
 '2019-05-05T00:00:00.000000000' '2019-05-06T00:00:00.000000000'
 '2019-05-07T00:00:00.000000000' '2019-05-08T00:00:00.000000000'
 '2019-05-09T00:00:00.000000000' '2019-05-10T00:00:00.000000000'
 '2019-05-11T00:00:00.000000000' '2019-05-12T00:00:00.000000000'
 '2019-05-13T00:00:00.000000000' '2019-05-14T00:00:00.000000000'
 '2019-05-15T00:00:00.000000000' '2019-05-16T00:00:00.000000000'
 '2019-05-17T00:00:00.000000000' '2019-05-20T00:00:00.000000000'
 '2019-05-21T00:00:00.000

In [23]:
import pandas as pd
import re

# Example DataFrames for sales_dataa and glance_viewsa
# Assume you have already loaded the actual data

# Clean special characters from SKU_NAME
# Removing special characters from SKU_NAME in both DataFrames
sales_data['cleaned_SKU_NAME'] = sales_data['SKU_NAME'].str.replace(r'[:;\\]', '', regex=True)
glance_views['cleaned_SKU_NAME'] = glance_views['SKU_NAME'].str.replace(r'[:;\\]', '', regex=True)

# Standardize the FEED_DATE to a common format (e.g., YYYY-MM-DD)
# Assuming FEED_DATE might be in different formats
sales_data['cleaned_FEED_DATE'] = pd.to_datetime(sales_data['FEED_DATE'], errors='coerce')
glance_views['cleaned_FEED_DATE'] = pd.to_datetime(glance_views['FEED_DATE'], errors='coerce')

# Display the cleaned data
print(sales_data[['SKU_NAME', 'cleaned_SKU_NAME', 'FEED_DATE', 'cleaned_FEED_DATE']].head())
print(glance_views[['SKU_NAME', 'cleaned_SKU_NAME', 'FEED_DATE', 'cleaned_FEED_DATE']].head())


     SKU_NAME cleaned_SKU_NAME  FEED_DATE cleaned_FEED_DATE
0  B12020KBUI       B12020KBUI 2019-05-18        2019-05-18
1  B12020KBUI       B12020KBUI 2019-05-19        2019-05-19
2  B12020KBUI       B12020KBUI 2019-05-22        2019-05-22
3  B12020KBUI       B12020KBUI 2019-05-23        2019-05-23
4  B12020KBUI       B12020KBUI 2019-05-27        2019-05-27
     SKU_NAME cleaned_SKU_NAME   FEED_DATE cleaned_FEED_DATE
0  B1212:PZ:V         B1212PZV  2019-05-01        2019-05-01
1  B1212:PZ:V         B1212PZV  2019-05-02        2019-05-02
2  B1212:PZ:V         B1212PZV  2019-05-03        2019-05-03
3  B1212:PZ:V         B1212PZV  2019-05-04        2019-05-04
4  B1212:PZ:V         B1212PZV  2019-05-05        2019-05-05


In [24]:
# Merging on cleaned_SKU_NAME and cleaned_FEED_DATE
merged_data = pd.merge(sales_data, glance_views, on=['cleaned_SKU_NAME', 'cleaned_FEED_DATE'], suffixes=('_sales', '_glance'))

# Display the merged data
print(merged_data.head())


   Unnamed: 0_sales SKU_NAME_sales FEED_DATE_sales     CATEGORY SUB_CATEGORY  \
0                 0     B12020KBUI      2019-05-18  1000 Inputs    1002 Mice   
1                 1     B12020KBUI      2019-05-19  1000 Inputs    1002 Mice   
2                 2     B12020KBUI      2019-05-22  1000 Inputs    1002 Mice   
3                 3     B12020KBUI      2019-05-23  1000 Inputs    1002 Mice   
4                 4     B12020KBUI      2019-05-27  1000 Inputs    1002 Mice   

   ORDERED_REVENUE  ORDERED_UNITS  REP_OOS cleaned_SKU_NAME cleaned_FEED_DATE  \
0              0.0            0.0      0.0       B12020KBUI        2019-05-18   
1              0.0            0.0      0.0       B12020KBUI        2019-05-19   
2              0.0            0.0      0.0       B12020KBUI        2019-05-22   
3              0.0            0.0      0.0       B12020KBUI        2019-05-23   
4              0.0            0.0      0.0       B12020KBUI        2019-05-27   

   Unnamed: 0_glance SKU_NAME_gl

In [25]:
# Export merged data for Tableau visualization
merged_data.to_csv('merged_sales_glance_data.csv', index=False)